In [2]:
import pandas as pd
import sqlite3


In [6]:
conn = sqlite3.connect('../unzippedData/im.db')

In [5]:
! ls ../unzippedData

bom.movie_gross.csv  rt.movie_info.tsv    tmdb.movies.csv
im.db                rt.reviews.tsv       tn.movie_budgets.csv


In [7]:
cursor = conn.cursor()

![Image of ERD Diagram IMDB](https://raw.githubusercontent.com/learn-co-curriculum/dsc-phase-2-project-v3/main/movie_data_erd.jpeg)


In [8]:
# displaying table

imdb_query = """ 
    SELECT *
    FROM sqlite_master
"""
imdb_df = pd.read_sql(imdb_query, conn)
imdb_df

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [12]:
# Connecting to Persons Table

persons_query = """ 
    SELECT *
    FROM persons
"""
persons_df = pd.read_sql(persons_query, conn)
persons_df.head()

,person_id,primary_name,birth_year,death_year,primary_profession
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator"


In [13]:
# Connecting to Persons Table

principals_query = """ 
    SELECT *
    FROM principals
"""
principals_df = pd.read_sql(principals_query, conn)
principals_df.head()

,movie_id,ordering,person_id,category,job,characters
0,tt0111414,1,nm0246005,actor,None,"[""The Man""]"
1,tt0111414,2,nm0398271,director,None,None
2,tt0111414,3,nm3739909,producer,producer,None
3,tt0323808,10,nm0059247,editor,None,None
4,tt0323808,1,nm3579312,actress,None,"[""Beth Boothby""]"


In [14]:
# Connecting to Known_For Table

known_for_query = """ 
    SELECT *
    FROM known_for
"""
known_for_df = pd.read_sql(known_for_query, conn)
known_for_df.head()

,person_id,movie_id
0,nm0061671,tt0837562
1,nm0061671,tt2398241
2,nm0061671,tt0844471
3,nm0061671,tt0118553
4,nm0061865,tt0896534


In [16]:
# Connecting to Movie Basics Table

movie_basics_query = """ 
    SELECT *
    FROM movie_basics
"""
movie_basics_df = pd.read_sql(movie_basics_query, conn)
movie_basics_df.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [21]:
# Connecting to Movie Ratings Table

movie_ratings_query = """ 
    SELECT *
    FROM movie_ratings
"""
movie_ratings_df = pd.read_sql(movie_ratings_query, conn)
movie_ratings_df

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21
...,...,...,...
73851,tt9805820,8.1,25
73852,tt9844256,7.5,24
73853,tt9851050,4.7,14
73854,tt9886934,7.0,5


In [ ]:
movie_ratings_query = """ 
    SELECT *
    FROM movie_ratings
    JOIN movie_basics
    
"""
movie_ratings_df = pd.read_sql(movie_ratings_query, conn)
movie_ratings_df

In [19]:
# Connecting to Movie Akas Table

movie_akas_query = """ 
    SELECT *
    FROM movie_akas
"""
movie_akas_df = pd.read_sql(movie_akas_query, conn)
movie_akas_df.head()

,movie_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,None,None,0.0
1,tt0369610,11,Jurashikku warudo,JP,None,imdbDisplay,None,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,None,imdbDisplay,None,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,None,None,short title,0.0
4,tt0369610,14,Jurassic World,FR,None,imdbDisplay,None,0.0


In [ ]:
# Make function to clean up column names
    # Title
    # Change underscore to space
    # In place 

In [20]:
# Make function to split up comma seperated values to become a set or list
    # will help easily do IN

In [ ]:
# profitability
# ratings * number_of_people

